In [12]:
import torch as th


th.nn.Module

class ArtificialHumanEnv():
    """
    Environment that runs the virtual humans and calculuates the value of the common good.

    Indices:
        t: agent types [0..1]
    """

    def __init__(
            self, *, contributors_model, n_agents, max_contribution, max_punishment, device):
        """
        Args:
            asdasd
        """
        self.device = device
        self.max_contribution = max_contribution
        self.max_punishment = max_punishment
        self.contributors_model = contributors_model
        self.n_agents = n_agents
        self.reset_state()


    def reset_state(self):
        self.state = {
            'punishments': th.zeros(self.n_agents, dtype=th.int16),
            'contributions': th.zeros(self.n_agents, dtype=th.int16),
            'payoffs': th.zeros(self.n_agents, dtype=th.float32),
            'valid': th.zeros(self.n_agents, dtype=th.bool),
            'common_good': th.zeros(1, dtype=th.float32),
            'episode_step': th.zeros(1, dtype=th.int16)
        }


    def __getattr__(self, name):
        if name in self.state:
            return self.state[name]
        else:
            raise AttributeError(f'Unkown state key {name}')

    def __setattr__(self, name, value):
        if isinstance(value, th.tensor):
            if name in self.state:
                self.state[name] = value
            else:
                raise AttributeError(f'Unkown state key {name}')

    @staticmethod
    def calc_common_good(contributions, punishments):
        return contributions.sum() * 1.6 + punishments.sum()

    @staticmethod
    def calc_payout(contributions, punishments, commond_good):
        # TODO: check how to handle missing values
        return 20 - contributions - punishments + 0.25 * commond_good

    def get_contributions(self):
        contributions = self.contributors_model.get_contributions(**self.state)
        return contributions

    def init_episode(self):
        self.episode += 1
        self.episode_step = 0
        self.contributions = self.get_contributions()
        return self.state

    def step(self, punishments):
        self.episode_step += 1

        assert punishments.max() <= self.max_punishment
        assert punishments.dtype == th.int64

        if (self.episode_step == self.episode_steps):
            done = True
        elif self.episode_step > self.episode_steps:
            raise ValueError('Environment is done already.')
        else:
            done = False

        self.punishment = punishments
        self.common_good = self.calc_common_good(self.contributions, self.punishments)
        self.payout = self.calc_payout(self.contributions, self.punishments, self.common_good)
        self.contributions = self.get_contributions()

        return self.state, self.common_good,  done


In [13]:

device = th.device('cpu')

env = ArtificialHumanEnv(
    contributors_model=None, n_agents=4, max_contribution=20, max_punishment=30, device=device)

RecursionError: maximum recursion depth exceeded

In [ ]:
env.test = 1